In [1]:
import os
os.chdir('D:\Deep Learning\Tensorflow & Keras\deploy2\project3')

In [2]:
%pwd

'D:\\Deep Learning\\Tensorflow & Keras\\deploy2\\project3'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path 

In [4]:
from TomatoDiseases.constant import *
from TomatoDiseases.utils.common import read_yaml, create_directories

In [5]:
class ConfigManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        
        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from TomatoDiseases import logger
from TomatoDiseases.utils.common import get_size

In [12]:
class DataIngest:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_filezip(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url= self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f'{filename} downloaded with the following info: {header}')
        else:
            logger.info(f'File already existed of size: {get_size(Path(self.config.local_data_file))}')
    
    def extract_zipfile(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingest = DataIngest(config=data_ingestion_config)
    data_ingest.download_filezip()
    data_ingest.extract_zipfile()
except Exception as e:
    raise e

[2023-07-10 21:20:28,511: INFO: common: yaml file: config.yaml loaded successfully]
[2023-07-10 21:20:28,512: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-10 21:20:28,513: INFO: common: created directory at: artifacts]
[2023-07-10 21:20:28,514: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-10 21:21:12,131: INFO: 3751046232: artifacts/data_ingestion/data.zip downloaded with the following info: Connection: close
Content-Length: 309459957
Content-Type: application/octet-stream
Content-MD5: 5geTUJtr4Isg40AqDpJA0A==
Last-Modified: Mon, 10 Jul 2023 06:33:02 GMT
ETag: "0x8DB810F83119F83"
Server: Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0
x-ms-request-id: cd63bb76-601e-0003-5a38-b30031000000
x-ms-version: 2020-04-08
x-ms-creation-time: Mon, 10 Jul 2023 06:33:02 GMT
x-ms-lease-status: unlocked
x-ms-lease-state: available
x-ms-blob-type: BlockBlob
Content-Disposition: attachment; filename=train.zip
x-ms-server-encrypted: true
Via: 1.1 varnish, 1.1